In [1]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy import signal
import datetime
import pickle
import time
%matplotlib inline

mozartの環境設定

In [2]:
#p_path="/home/takeyama/pywork/ipython/2016-05-30/"

In [3]:
#cd ~/Documents/SyncRecord/cleaning-addingLABEL/

workstationの環境設定

In [4]:
cd ~/Documents/AddingLabel+FFTdata/

/home/takeyama/Documents/AddingLabel+FFTdata


In [5]:
class SensorData:
    
    def __init__(self):
        print "__class__"
        # raw data
        self._RawData={}    
        # fft data
        self._FFTData={}
        # power spectol data
        self._PowerData={}    
        # flag exsist data
        self._Flag_exist_data=False
        
        self._columns=['AccX','AccY','AccZ','GyrX','GyrY','GyrZ']
        self._fft_col=['fft_AccX','fft_AccY','fft_AccZ','fft_GyrX','fft_GyrY','fft_GyrZ']
        self._power_col=['power_AccX','power_AccY','power_AccZ','power_GyrX','power_GyrY','power_GyrZ']
        
    def ImportCSV(self,Sclass,csv_file):
        self.ClassName=Sclass
        self.Flag_exist_data=True
        # design dataframe
        data = pd.read_csv(csv_file,encoding="SHIFT-JIS")
        data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
        data.Time=pd.to_datetime(data.Time)
        data = pd.pivot_table(data,values=[u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ'],index=[u'Time'])
        
        # convert numpy.darray
        AccX=data.AccX.values*0.0001
        AccY=data.AccY.values*0.0001
        AccZ=data.AccZ.values*0.0001
        GyrX=data.GyrX.values*0.01
        GyrY=data.GyrY.values*0.01
        GyrZ=data.GyrZ.values*0.01
        Time=data.index.to_pydatetime().astype('datetime64[ns]')
        
        # regist each raw data 
        self._RawData['AccX'] = AccX
        self._RawData['AccY'] = AccY
        self._RawData['AccZ'] = AccZ
        self._RawData['GyrX'] = GyrX
        self._RawData['GyrY'] = GyrY
        self._RawData['GyrZ'] = GyrZ
        self._RawData['Time'] = Time
    
    def ShowFlagExistData(self):
        return self.Flag_exist_data
    
    def GetColumns(self):
        return self._columns
    
    def GetTime(self):
        return self._RawData['Time']

    def ShowAllDf(self):
        print 'AccX : ';print self._RawData['AccX']
        print 'AccY : ';print self._RawData['AccY']
        print 'AccZ : ';print self._RawData['AccZ']
        print 'GyrX : ';print self._RawData['GyrX']
        print 'GyrY : ';print self._RawData['GyrY']
        print 'GyrZ : ';print self._RawData['GyrZ']
        
    def _Time2Num(self,time):
        return np.where(self._RawData['Time']==np.datetime64(time) )[0][0]
    
    def ShowQuery(self,Sname,rng=[]):
        data = self._RawData[Sname]
        print Sname+':'+str( data[rng[0]:rng[1]])
            
    def _sliding_window(self,Sname,samp, overlap):
        count =0
        s =self._RawData['Time'][0]
        start=self._Time2Num(s)
        g = s+np.timedelta64(samp*10,'ms')
        goal= self._Time2Num(g)
        yield self._RawData[Sname][start:goal]
        
        add=overlap*0.01
        
        while True:
            try:
                count +=1
                s =s+np.timedelta64(samp*10,'ms')
                start=self._Time2Num(s)
                g  =s+np.timedelta64(samp*10,'ms')
                goal= self._Time2Num(g)
                yield self._RawData[Sname][start:goal]
            except StopIteration:
                print '_sliding_window StopIteration'
                break   
            except IndexError:
                print '_sliding_window IndexError'
                break  
                
# Fast Frier transaction            
    def GetFFT(self,Sfft,samp):
        return np.load(p_path+'fft/'+self.ClassName+'_'+Sfft+'_'+str(samp)+'.npz')['arr_0']
    
    def CalcFFT(self,samp,overlap=0.5):
        start = time.time()
        fft_data = np.array([])
        
        for n,f in zip( self._columns,self._fft_col):
            print 'start'+n+'->'+f
            sw = self._sliding_window(n,samp,overlap)
            while True:
                try:
                    d=sw.next()
                    fft_data = np.append(fft_data, fftpack.fft(d)[1:(samp/2)+1] )   # fftの直流成分を除くsample/2の
                except StopIteration:
                    print 'CalcFFTStopIteration'
                    fft_data = fft_data.reshape(len(fft_data)/(samp/2),(samp/2) )
                    self._FFTData[f] = fft_data
                    np.savez(p_path+'fft/'+self.ClassName+'_'+str(f)+'_'+str(samp),self._FFTData[f])
                    break   
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"
           

# Spectol Power 
    def GetPower(self,Spower,samp):
        return np.load(p_path+'power/'+self.ClassName+'_'+Spower+'_'+str(samp)+'.npz')['arr_0']
    
    def _power(self,fft_array):
        p=lambda x,y : np.sqrt(x**2+y**2)
        power_array=np.array([])
        for vector in fft_array:
            tmp = p( np.real(vector),np.imag(vector) )
            tmp = tmp/np.sum(tmp)
            power_array = np.append(power_array,tmp)
        return power_array
        
    def CalcPower(self,samp,overlap=0.5):
        start = time.time()
        for fft_name,power_name in zip( self._fft_col, self._power_col):
            print 'start'+fft_name+'->'+power_name
            fft_data = self.GetFFT(fft_name,samp)
            power_data=self._power(fft_data)
            np.savez(p_path+'power/'+self.ClassName+'_'+power_name+'_'+str(samp),power_data)
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"

センサーのスタート時間を求める

In [6]:
data1 = SensorData()
data2 = SensorData()
data3 = SensorData()
data4 = SensorData()
data5 = SensorData()

__class__
__class__
__class__
__class__
__class__


In [7]:
data1.ImportCSV('left_hand','Conv-left-hand-ags.csv')
data2.ImportCSV('left_leg','Conv-left-leg-ags.csv')
data3.ImportCSV('right_hand','Conv-right-hand-ags.csv')
data4.ImportCSV('right_leg','Conv-right-leg-ags.csv')
data5.ImportCSV('west','Conv-west-ags.csv')

In [8]:
time1=data1.GetTime()
time2=data2.GetTime()
time3=data3.GetTime()
time4=data4.GetTime()
time5=data5.GetTime()

In [9]:
if min(time1) < min(time2):
    MAX = min(time2)
else:
    MAX = min(time1)

if MAX < min(time3):
    MAX = min(time3)

if MAX < min(time4):
    MAX = min(time4)

if MAX < min(time5):
    MAX = min(time5)

print MAX

2016-06-08T18:06:15.877000000


Conv-から始まるファイルはすでにhh:mm:ssSSのように変換されている。
しかし、この形では四捨五入といった丸めることができない。

よって、最初のmsの状態で位置の桁を四捨五入すれば簡単に丸めることができるのではないだろうか？
試しに、値をleft-hand.csvの時刻から四捨五入して、hh:mm:ssSSの形に直してみる。

In [10]:
np.array(55844456,dtype='datetime64[ms]')

array(datetime.datetime(1970, 1, 1, 15, 30, 44, 456000), dtype='datetime64[ms]')

In [11]:
np.datetime64(55844456,'ms')

numpy.datetime64('1970-01-01T15:30:44.456')

In [12]:
round(55844456*0.1)*10

55844460.0

In [13]:
a=round(55844456*0.1)*10
np.datetime64(int(a),'ms')

numpy.datetime64('1970-01-01T15:30:44.460')

In [14]:
np.datetime64(55844456,'ms')

numpy.datetime64('1970-01-01T15:30:44.456')

CSVファイルをnumpyで読み込んでクラスのインポートと同じような動作ができるかどうかやってみる。

In [15]:
time = np.genfromtxt("left-hand.csv",delimiter=",",usecols=(1)) #時間の列だけを抽出

In [16]:
func = lambda x: int(round(x*0.1)*10)
a= map( func, time) 

t = lambda x: np.datetime64(x,'ms')
times = map(t, a)
time = np.array(times,dtype=np.datetime64)

In [17]:
time[0]

numpy.datetime64('1970-01-01T18:06:15.700')

In [18]:
np.datetime64(55844456,'ms')

numpy.datetime64('1970-01-01T15:30:44.456')

前処理クラス　時系列の統一　アルゴリズム　考案　2016-06-07

現在の作業状況の途中経過をここにメモする。
まずは、肝心の時系列だがpandasで読み込むよりnumpy.loadcsv使ったほうが正確である
しかし、この読み込み方法だと問題が１つある。それは、加速度データがなぜかよみこまれない
からである。
一番怪しいのは、マイナスとか符号が原因でエラーが起きているのではないかと考える。


In [19]:
#sampling freqency 
samp=10

サンプリング周波数を決め、切り上げ、切り捨て、四捨五入を関数を使用しないで実装してみる
この計算では各データの単位はmsで行う。
理由として、csvファイルの時刻は当日０時から経過したmsであるから

In [20]:
time = np.genfromtxt("left-hand.csv",delimiter=",",usecols=(1)) #時間の列だけを抽出

In [21]:
time

array([ 65175696.,  65175706.,  65175716., ...,  65730376.,  65730386.,
        65730396.])

In [22]:
#切り捨て
output = (time/samp).astype(int)*samp
output

array([65175690, 65175700, 65175710, ..., 65730370, 65730380, 65730390])

In [23]:
#切り上げ
output = ((time+samp)/samp).astype(int)*samp
output

array([65175700, 65175710, 65175720, ..., 65730380, 65730390, 65730400])

In [24]:
#四捨五入
output = ( (time+samp/2)/samp).astype(int)*samp
output

array([65175700, 65175710, 65175720, ..., 65730380, 65730390, 65730400])

結局、最初のデータは時間を指すことはわかるが、２番めの数字は元データをみても該当するものはない。
よって、２通りのCSV読む方法を取る。１つはpandas、もう１つはnumpy.loadtxtである。
pandasのほうでは、加速度、角速度を読み込む
numpyのほうでは、時系列を読み込む

In [25]:
'''
csv_file -> ファイル名
mode Round -> 四捨五入
     Roundup -> 切り上げ
     Rounddown -> 切り捨て
'''
def ImportCSV(csv_file,samp,mode='Round'):
        # data dictionary 
        RawData={}   
        
        # design dataframe and import csv
        data = pd.read_csv(csv_file,encoding="SHIFT-JIS")
        data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
        data = data[ data['Type']=='ags']
        
        # convert numpy.darray 
        # Acc Data  [0.1mG]=>[G]
        # Gyr Data  [0.01dps]=>[dps]   ...dps=degree per second
        AccX=data.AccX.values*0.0001
        AccY=data.AccY.values*0.0001
        AccZ=data.AccZ.values*0.0001
        GyrX=data.GyrX.values*0.01
        GyrY=data.GyrY.values*0.01
        GyrZ=data.GyrZ.values*0.01
        
        # regist each raw data 
        RawData['AccX'] = AccX
        RawData['AccY'] = AccY
        RawData['AccZ'] = AccZ
        RawData['GyrX'] = GyrX
        RawData['GyrY'] = GyrY
        RawData['GyrZ'] = GyrZ
        
        # import time by using numpy
        time = data.Time.values #時間の列だけを抽出       
        
        if mode == 'Roundup':
            func = lambda x: int(x/samp)*samp
        elif mode == 'Rounddown':
            func = lambda x: int(x/samp)*samp
        elif mode == 'Round':
            func = lambda x: int((x+samp/2)/samp)*samp
        #ERROR
        else:
            print 'check mode and inputed word is caused error'
            return -1
        
        output = map(func,time)
        RawData['Time'] = output
        return RawData
            

In [26]:
check1 = ImportCSV('left-hand.csv',10)

In [27]:
check1['Time'][66]

65176370

この関数を使用することによって、切り捨て切り上げ、四捨五入を行い
時系列の時間を10msごとのデータになった ->1の桁はゼロである

次に、各データの時系列を同期する

In [28]:
check2 = ImportCSV('left-leg.csv',10)
check3 = ImportCSV('right-hand.csv',10)
check4 = ImportCSV('right-leg.csv',10)
check5 = ImportCSV('west.csv',10)

センサの時系列の最小値を求める。そして、それぞれの最小値を比較して
その中で一番大きい値をスタート時間とする

In [29]:
if min(check1['Time']) < min(check2['Time']):
    MAX = min(check2['Time'])
else:
    MAX = min(check1['Time'])

if MAX < min(check3['Time']):
    MAX = min(check3['Time'])

if MAX < min(check4['Time']):
    MAX = min(check4['Time'])

if MAX < min(check5['Time']):
    MAX = min(check5['Time'])

print MAX

65175890


In [30]:
print min(check1['Time'])
print min(check2['Time'])
print min(check3['Time'])
print min(check4['Time'])
print min(check5['Time'])

65175710
65175890
65175670
65175810
65175760


センサの時系列の最大値を求める。そして、それぞれの最大値を比較して
その中で一番小さい値をスタート時間とする

In [31]:
if max(check1['Time']) < max(check2['Time']):
    MIN = max(check1['Time'])
else:
    MIN = max(check2['Time'])

if MIN > max(check3['Time']):
    MIN = max(check3['Time'])

if MIN > max(check4['Time']):
    MIN = max(check4['Time'])

if MIN > max(check5['Time']):
    MIN = max(check5['Time'])

print MIN

65730150


In [32]:
print max(check1['Time'])
print max(check2['Time'])
print max(check3['Time'])
print max(check4['Time'])
print max(check5['Time'])

65730400
65730970
65730150
65730810
65730620


各センサの時系列データをスタート時間を０番目となるようにする


In [33]:
for i in range(0, len(check1['Time'])):
    if 65175890 == check1['Time'][i]:
            print 'start index number is '+str(i)

start index number is 18


In [34]:
check1['Time'][18]

65175890

In [35]:
for i in range(0, len(check2['Time'])):
    if 65175890 == check2['Time'][i]:
            print 'start index number is '+str(i)

start index number is 0


In [36]:
check2['Time'][0]

65175890

In [37]:
for i in range(0, len(check3['Time'])):
    if 65175890 == check3['Time'][i]:
            print 'start index number is '+str(i)

start index number is 22


In [38]:
check3['Time'][22]

65175890

In [39]:
for i in range(0, len(check4['Time'])):
    if 65175890 == check4['Time'][i]:
            print 'start index number is '+str(i)

start index number is 8


In [40]:
check4['Time'][8]

65175890

In [41]:
for i in range(0, len(check5['Time'])):
    if 65175890 == check5['Time'][i]:
            print 'start index number is '+str(i)

start index number is 13


In [42]:
check5['Time'][13]

65175890

上記の方法で求めたスタート時間とゴール時間の間を扱う。
しかし、その区間にもセンサによっては取得できてない可能性もあるので確認する

手法としては、スタート時間からゴール時間をステップ+10とした連続値の配列を作成する
各センサデータの時系列はこの配列（以後、参照配列と呼称する）と比較してNANがあるかどうか調べる
NANがある場合、同じ大きさの配列のフラグにFalseを入れる。ない場合はTrueを入れる


In [70]:
ReferenceIndex = range(65175890,65730150,10)
Flag_ExistData = {}
Flag_ExistData['check1']= range(0, len(ReferenceIndex))
Flag_ExistData['check2']= range(0, len(ReferenceIndex))
Flag_ExistData['check3']= range(0, len(ReferenceIndex))
Flag_ExistData['check4']= range(0, len(ReferenceIndex))
Flag_ExistData['check5']= range(0, len(ReferenceIndex))

2016-6-8
以後、行われていることはReferenceIndexと各データcheck1-5で違う時間
をもっているかどうか
つまり、スタート時間からゴール時間のデータを各データは保持している
かどうかを調査する

In [65]:
count = 0
checkData = check1['Time']
i=0
for i in range(0, len(checkData)):
    if 65175890 == checkData[i]:
        startNum=i
        print 'start index number is '+str(i)
        break
        
if i == len(checkData)-1:
    startNum=0
    print 'start index number is '+str(0)
    
for i in range(0, len(ReferenceIndex)):
    if (checkData[i+startNum]==ReferenceIndex[i]):
        #print str(i)+' is all sensor data complete'
        Flag_ExistData['check1'][i]=True
    else:
        #print str(i)+' is data has nan data'
        Flag_ExistData['check1'][i]=False
        count +=1
print count

start index number is 18
0


In [66]:
count = 0
checkData = check2['Time']
i=0
for i in range(0, len(checkData)):
    if 65175890 == checkData[i]:
        startNum=i
        print 'start index number is '+str(i)
        break
        
if i == len(checkData)-1:
    startNum=0
    print 'start index number is '+str(0)
    
for i in range(0, len(ReferenceIndex)):
    if (checkData[i+startNum]==ReferenceIndex[i]):
        #print str(i)+' is all sensor data complete'
        Flag_ExistData['check2'][i]=True
    else:
        #print str(i)+' is data has nan data'
        Flag_ExistData['check2'][i]=False
        count +=1
print count

start index number is 0
0


In [67]:
count = 0
checkData = check3['Time']
i=0
for i in range(0, len(checkData)):
    if 65175890 == checkData[i]:
        startNum=i
        print 'start index number is '+str(i)
        break
        
if i == len(checkData)-1:
    startNum=0
    print 'start index number is '+str(0)
    
for i in range(0, len(ReferenceIndex)):
    if (checkData[i+startNum]==ReferenceIndex[i]):
        #print str(i)+' is all sensor data complete'
        Flag_ExistData['check3'][i]=True
    else:
        #print str(i)+' is data has nan data'
        Flag_ExistData['check3'][i]=False
        count +=1
print count

start index number is 22
0


In [68]:
count = 0
checkData = check4['Time']
i=0
for i in range(0, len(checkData)):
    if 65175890 == checkData[i]:
        startNum=i
        print 'start index number is '+str(i)
        break
        
if i == len(checkData)-1:
    startNum=0
    print 'start index number is '+str(0)
    
for i in range(0, len(ReferenceIndex)):
    if (checkData[i+startNum]==ReferenceIndex[i]):
        #print str(i)+' is all sensor data complete'
        Flag_ExistData['check4'][i]=True
    else:
        #print str(i)+' is data has nan data'
        Flag_ExistData['check4'][i]=False
        count +=1
print count

start index number is 8
0


In [69]:
count = 0
checkData = check5['Time']
i=0
for i in range(0, len(checkData)):
    if 65175890 == checkData[i]:
        startNum=i
        print 'start index number is '+str(i)
        break
        
if i == len(checkData)-1:
    startNum=0
    print 'start index number is '+str(0)

for i in range(0, 55426):
    print checkData[i+startNum]
    if (checkData[i+startNum]==ReferenceIndex[i]):
        #print str(i)+' is all sensor data complete'
        Flag_ExistData['check5'][i]=True
    else:
        #print str(i)+' is data has nan data'
        Flag_ExistData['check5'][i]=False
        count +=1
print count

start index number is 13
65175890
65175900
65175910
65175920
65175930
65175940
65175950
65175960
65175970
65175980
65175990
65176000
65176010
65176020
65176030
65176040
65176050
65176060
65176070
65176080
65176090
65176100
65176110
65176120
65176130
65176140
65176150
65176160
65176170
65176180
65176190
65176200
65176210
65176220
65176230
65176240
65176250
65176260
65176270
65176280
65176290
65176300
65176310
65176320
65176330
65176340
65176350
65176360
65176370
65176380
65176390
65176400
65176410
65176420
65176430
65176440
65176450
65176460
65176470
65176480
65176490
65176500
65176510
65176520
65176530
65176540
65176550
65176560
65176570
65176580
65176590
65176600
65176610
65176620
65176630
65176640
65176650
65176660
65176670
65176680
65176690
65176700
65176710
65176720
65176730
65176740
65176750
65176760
65176770
65176780
65176790
65176800
65176810
65176820
65176830
65176840
65176850
65176860
65176870
65176880
65176890
65176900
65176910
65176920
65176930
65176940
65176950
65176960
651

IndexError: list index out of range

上のアルゴリズムのについての原因を考える。原因は、check5の配列の長さが54992あるのに対して
ReferenceIndexの長さは55426である。共通区間はあくまでも各データの時系列のデータの長さより小さくなければ
共通という意味に矛盾する。つまり、ここが原因なのではないかと考える。

確認方法として、配列の前後の差を調べる。そして、差が10以外のデータのインデックスを探す

In [89]:
checkData = check1['Time']
diff =[]
diff_index=[]
for i in range(0,len(checkData)-1):
    if ( checkData[i+1]-checkData[i] )!=10:
        diff.append(checkData[i+1]-checkData[i])
        diff_index.append(i)
        print 'Error Index Num='+str(i)+' diff='+str( checkData[i+1]-checkData[i] )
        print str(i)+'s Data ='+str( checkData[i])
        print str(i+1)+'s Data ='+str( checkData[i+1])

In [91]:
checkData = check2['Time']
diff =[]
diff_index=[]
for i in range(0,len(checkData)-1):
    if ( checkData[i+1]-checkData[i] )!=10:
        diff.append(checkData[i+1]-checkData[i])
        diff_index.append(i)
        print 'Error Index Num='+str(i)+' diff='+str( checkData[i+1]-checkData[i] )
        print str(i)+'s Data ='+str( checkData[i])
        print str(i+1)+'s Data ='+str( checkData[i+1])

In [92]:
checkData = check3['Time']
diff =[]
diff_index=[]
for i in range(0,len(checkData)-1):
    if ( checkData[i+1]-checkData[i] )!=10:
        diff.append(checkData[i+1]-checkData[i])
        diff_index.append(i)
        print 'Error Index Num='+str(i)+' diff='+str( checkData[i+1]-checkData[i] )
        print str(i)+'s Data ='+str( checkData[i])
        print str(i+1)+'s Data ='+str( checkData[i+1])

In [93]:
checkData = check4['Time']
diff =[]
diff_index=[]
for i in range(0,len(checkData)-1):
    if ( checkData[i+1]-checkData[i] )!=10:
        diff.append(checkData[i+1]-checkData[i])
        diff_index.append(i)
        print 'Error Index Num='+str(i)+' diff='+str( checkData[i+1]-checkData[i] )
        print str(i)+'s Data ='+str( checkData[i])
        print str(i+1)+'s Data ='+str( checkData[i+1])

In [98]:
# zero patting
checkData = check5['Time']
diff =[]
diff_index=[]
for i in range(0,len(checkData)-1):
    if ( checkData[i+1]-checkData[i] )!=10:
        diff.append(checkData[i+1]-checkData[i])
        diff_index.append(i)
        print 'Error Index Num='+str(i)+' diff='+str( checkData[i+1]-checkData[i] )
        print str(i)+'s Data ='+str( checkData[i])
        print str(i+1)+'s Data ='+str( checkData[i+1])

start = 0
even = 0
tmp = np.array([])
if len(diff)!=0:
    for count,l in enumerate(diff_index):
        # index の数が偶数のとき、偶数番目のdiff_indexの値をevenに格納
        if count%2 ==0:
            even = l
            tmp = np.append(tmp,checkData[start:l])
        # index の数が奇数のとき、evenと奇数番目のdiff_indexの値の区間をNANを格納する
        else:
            for i in range(even,l):
                tmp = np.append(tmp,np.nan)
                
tmp = np.append(tmp, checkData[])
print len(tmp)

Error Index Num=5589 diff=1330
5589s Data =65231650
5590s Data =65232980
Error Index Num=5650 diff=3000
5650s Data =65233580
5651s Data =65236580
Error Index Num=5813 diff=90
5813s Data =65238200
5814s Data =65238290
Error Index Num=5870 diff=430
5870s Data =65238850
5871s Data =65239280
Error Index Num=6266 diff=140
6266s Data =65243230
6267s Data =65243370
Error Index Num=6302 diff=20
6302s Data =65243720
6303s Data =65243740
17822


In [96]:
4%2

0

[1330, 3000, 90, 430, 140, 20]

In [72]:
len(check2['Time'])

55509

In [73]:
len(check3['Time'])

55449

In [74]:
len(check4['Time'])

55501

In [75]:
len(check5['Time'])

54992

In [58]:
len(ReferenceIndex)

55426

In [ ]:
ReferenceIndex[0]

In [ ]:
check2['Time'][0] 

具体的なnumpyのloadtxt,genfromtxtの違い

np.loadtxt('left-hand.csv',delimiter=',',usecols=(2,3,4))

np.genfromtxt('left-hand.csv',delimiter=',',usecols=(2,3,4))